In [2]:
cd ../..

/Users/ericpei/Documents/GalvClas/fantasy-score-predictor


In [3]:
from scripts import make_df

In [4]:
from scripts import random_forest

/Users/ericpei/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [857]:
#make_df.create_csvs('alldata')

In [858]:
df = make_df.create_df()

In [859]:
df = make_df.clean_df(df)

scripts/make_df.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df['FG']=(np.array(df['FG'])).astype(int)


In [860]:
df = make_df.drop_zerominutes(df)

In [861]:
df = make_df.add_features(df)

In [862]:
#look at this later
Player_Averages = make_df.make_averages(df)

In [863]:
date_matrix = make_df.get_date_matrix(df)

In [864]:
make_df.addcolumns(make_df.make_lists(date_matrix), df)

In [865]:
tp_matrix = df[['Player Name', '3P']].values
fg_matrix = df[['Player Name', 'FG']].values
ft_matrix = df[['Player Name', 'FT']].values
reb_matrix = df[['Player Name', 'TRB']].values
ast_matrix = df[['Player Name', 'AST']].values
blk_matrix = df[['Player Name', 'BLK']].values
stl_matrix = df[['Player Name', 'STL']].values
tov_matrix = df[['Player Name', 'TOV']].values
usg_matrix = df[['Player Name', 'USG%']].values
ftr_matrix = df[['Player Name', 'FTr']].values
pm_matrix = df[['Player Name', '+/-']].values
ts_matrix = df[['Player Name', 'TS%']].values
pf_matrix = df[['Player Name', 'PF']].values
ortg_matrix = df[['Player Name', 'ORtg']].values
drtg_matrix = df[['Player Name', 'DRtg']].values

In [866]:
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=tp_matrix, column='3P'), df, name='TP')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=fg_matrix, column='FG'), df, name='FG')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=ft_matrix, column='FT'), df, name='FT')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=reb_matrix, column='TRB'), df, name='TRB')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=ast_matrix, column='AST'), df, name='AST')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=blk_matrix, column='BLK'), df, name='BLK')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=stl_matrix, column='STL'), df, name='STL')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=tov_matrix, column='TOV'), df, name='TOV')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=usg_matrix, column='USG%'), df, name ='USGP')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=ftr_matrix, column='FTr'), df, name ='FTr')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=pm_matrix, column='+/-'), df, name ='PM')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=ts_matrix, column='TS%'), df, name ='TSP')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=pf_matrix, column='PF'), df, name ='PF')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=ortg_matrix, column='ORtg'), df, name ='ORtg')
make_df.add_slag_columns(make_df.make_stat_lists(Player_Averages, matrix=drtg_matrix, column='DRtg'), df, name ='DRtg')

In [867]:
from scripts import elevation_flight

In [868]:
Elevation_Lookup = elevation_flight.read_to_dict()

In [869]:
elevation_flight.lookup_and_add(Elevation_Lookup, df)

# Exploration

In [870]:
from sklearn.metrics import mean_squared_error

In [871]:
from sklearn.ensemble import RandomForestRegressor

### Make a testing Set

In [872]:
forest_df, y_df = random_forest.date_with_forest(df, remove_date=True)

In [873]:
test_df, testy_df = random_forest.date_with_forest(df, remove_date=False)

In [874]:
import datetime

In [875]:
today = datetime.date.today()
stop_average_date = today - datetime.timedelta(0)

In [876]:
test_df = test_df[test_df['Date'] > stop_average_date]
testy_df = testy_df[testy_df['Date'] > stop_average_date]

In [877]:
test_df = test_df.ix[:,test_df.columns != 'Date']
testy_df = testy_df.ix[:, testy_df.columns != 'Date']

In [878]:
#forest_df, y_df = random_forest.prepare_for_forest(df)

In [879]:
TestSubject = 'Stephen Curry'

In [880]:
Stephcurry, Stephcurryafter_X, Stephcurryafter_y = random_forest.individual_forest_df(test_df, testy_df, TestSubject)

In [881]:
Stephcurry, Stephcurry_X, Stephcurry_y = random_forest.individual_forest_df(forest_df, y_df, TestSubject)

In [882]:
X_train, X_test, y_train, y_test = random_forest.crossval_player(Stephcurry, forest_df[forest_df["Player Name"] != TestSubject], y_df[y_df['Player Name'] != TestSubject], cat=2)
X_trainA, X_testA, y_trainA, y_testA = random_forest.crossval_player(Stephcurry, test_df, testy_df, cat=2)
X_trainS, X_testS, y_trainS, y_testS = random_forest.crossval_player(Stephcurry, Stephcurry_X, Stephcurry_y, cat=2)
X_trainSA, X_testSA, y_trainSA, y_testSA = random_forest.crossval_player(Stephcurry, Stephcurryafter_X, Stephcurryafter_y, cat = 2)

scripts/random_forest.py:85: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  model.predict(y[cat])


###### X Train is the df without Curry
###### X Train2 is the df with just Curry
###### S = Curry A is after cutoff

In [883]:
X_trainfull = pd.concat([X_train, X_trainS, X_trainS, X_trainSA])
y_trainfull = pd.concat([y_train, y_trainS, y_trainS, y_trainSA])

Now X Train2 is just Curry, while X Train is everyone including Curry

In [884]:
everyFGmean = pd.concat([y_train, y_trainS]).mean()
stephFGmean = y_trainS.mean()

In [885]:
len(X_test), len(X_testSA)

(68469, 0)

In [886]:
benchmark = np.full(len(X_testA)+len(X_testSA), everyFGmean)
s_benchmark = np.full(len(X_testSA), stephFGmean)

In [887]:
rf = RandomForestRegressor()
rf2 = RandomForestRegressor()

In [888]:
rf.fit(X_trainfull, y_trainfull)
rf2.fit(X_trainS, y_trainS)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

# Try with original 8

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
standard_scaler = StandardScaler()

In [54]:
scaled = standard_scaler.fit_transform(PA48.iloc[:,:8])

In [55]:
km.fit(scaled)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=5, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=30, tol=0.0001,
    verbose=0)

In [56]:
dict_nonpca = pca_ready.make_cluster_dictionary2(km, PA48.iloc[:,:8])

In [57]:
dict_nonpca[0][:10]

['A.J. Price',
 'Aaron Brooks',
 'Adam Morrison',
 'Adonis Thomas',
 'Al Harrington',
 'Alan Anderson',
 'Alando Tucker',
 'Alex Acker',
 'Allan Ray',
 'Allen Crabbe']

# 3D Plot on PCA

In [58]:
from mpl_toolkits.mplot3d import Axes3D

In [59]:
from scripts import makeplot

In [66]:
mat.sort_values(by=0, ascending=True).head()

,0,1,2
235,-2.098383,1.210921,1.032102
1009,-2.056424,-0.732874,-2.048537
1007,-1.962149,2.345746,-2.531527
275,-1.873621,1.307856,-3.433516
1067,-1.667683,-1.378243,0.427694


In [67]:
PA48.index[[1086, 147, 551, 1041, 1063, 21, 671]]

Index([u'Thanasis Antetokounmpo', u'Bryon Russell', u'Jerrelle Benimon',
       u'Sim Bhullar', u'Steven Hill', u'Alan Williams', u'Kevon Looney'],
      dtype='object', name=u'Player Name')

#### Points 1061, 147, 550, 1039, 1084 are problematic

# Find outliers that are messing up PCA

### Drop them from PCA

In [68]:
PA48_nout = PA48.drop(PA48.index[[1086, 147, 551, 1041, 1063, 21, 671]])

In [69]:
mat_nout, comp_nout = pca_ready.run_pca(PA48_nout.iloc[:,:8])

In [70]:
km = KMeans(n_clusters=3, random_state = 30)

In [71]:
km.fit(mat_nout)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=3, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=30, tol=0.0001,
    verbose=0)

In [72]:
dict_pca2_nout = pca_ready.make_cluster_dictionary2(km, PA48_nout.iloc[:,:8])

In [73]:
dict_pca2_nout

defaultdict(list,
            {0: ['A.J. Price',
              'Aaron Brooks',
              'Aaron McKie',
              'Aaron Miles',
              'Acie Law',
              'Alexey Shved',
              'Allen Iverson',
              'Alvin Williams',
              'Andre Barrett',
              'Andre Iguodala',
              'Andre Miller',
              'Andrei Kirilenko',
              'Anfernee Hardaway',
              'Anthony Carter',
              'Anthony Johnson',
              'Antonio Burks',
              'Antonio Daniels',
              'Armon Johnson',
              'Baron Davis',
              'Ben Hansbrough',
              'Ben Uzoh',
              'Beno Udrih',
              'Bobby Brown',
              'Boris Diaw',
              'Brandon Jennings',
              'Brandon Knight',
              'Brandon Roy',
              'Brevin Knight',
              'Brian Roberts',
              'Briante Weber',
              'C.J. Watson',
              'Cameron Payne',
  

# Silhouette Score Benchmark

In [75]:
from scripts import makesilplot

##### lets look at the actual clusters

In [890]:
dict_pca4_nout = pca_ready.make_cluster_dictionary2(km, PA48_nout)
dict_pca4_nout[2][:10]

['Aaron Harrison',
 'Adam Morrison',
 'Adonis Thomas',
 'Al Harrington',
 'Al Thornton',
 'Alan Anderson',
 'Alando Tucker',
 'Alec Burks',
 'Alex Acker',
 'Alex Kirk']

#### despite silhouette scores indicating clusters not optimal, clusters seem to make sense. This cluster has a very high density of guards. Steph Curry, Chris Paul, Steve Nash, Russell Westbrook, Tony Parker, Jeremy Lin.

In [649]:
len(df)

274005

# Cluster Sense

### Let us evaluate these clusters by looking at the 10 highest expected scores per cluster

In [78]:
player_score_list1 = []
for player in dict_pca4_nout[0]:
    player_score_list1.append((player, PA.ix[player, 'Score']))
sorted(player_score_list1, key=lambda x: x[1], reverse=True)[:10]

[('LeBron James', 48.513389626055492),
 ('Kobe Bryant', 41.45146036161335),
 ('Chris Paul', 41.370413436692509),
 ('Dwyane Wade', 40.589415041782729),
 ('Russell Westbrook', 39.889455782312922),
 ('Allen Iverson', 39.085526315789465),
 ('Stephen Curry', 38.701008064516131),
 ('John Wall', 37.757582938388616),
 ('Damian Lillard', 34.937577639751559),
 ('James Harden', 34.71102803738318)]

In [79]:
player_score_list2 = []
for player in dict_pca4_nout[1]:
    player_score_list2.append((player, PA.ix[player, 'Score']))
sorted(player_score_list2, key=lambda x: x[1], reverse=True)[:10]

[('Kevin Durant', 42.536760124610595),
 ('Anthony Davis', 40.734999999999999),
 ('Blake Griffin', 39.678345498783457),
 ('Dwight Howard', 39.317146433041302),
 ('Carmelo Anthony', 39.157449664429542),
 ('DeMarcus Cousins', 38.941204819277111),
 ('Yao Ming', 38.010330578512395),
 ('Pau Gasol', 37.757727873183619),
 ('Dirk Nowitzki', 36.487301587301587),
 ('Karl-Anthony Towns', 36.335365853658537)]

In [80]:
player_score_list3 = []
for player in dict_pca4_nout[2]:
    player_score_list3.append((player, PA.ix[player, 'Score']))
sorted(player_score_list3, key=lambda x: x[1], reverse=True)[:10]

[('Rudy Gay', 30.728769017980639),
 ('Joe Johnson', 30.083922046285014),
 ('Michael Redd', 28.352050473186114),
 ('Luol Deng', 28.300786369593705),
 ('Andrew Wiggins', 27.757055214723927),
 ('Jason Richardson', 27.75009174311927),
 ('DeMar DeRozan', 27.351532567049812),
 ('Danny Granger', 27.222013651877131),
 ('Stephen Jackson', 27.045886654478981),
 ('Klay Thompson', 26.98268733850129)]

# Cluster Two

In [83]:
from scripts import random_forest

In [85]:
from scripts import clusters

In [86]:
PA140 = make_df.make_averages(df)

In [93]:
model_dict2 = clusters.get_model_dict(df, cluster_dict=dict_pca4_nout, cluster_num=2)

# Cluster One

In [ ]:
from scripts import clusters

In [94]:
model_dict1 = clusters.get_model_dict(df, cluster_dict=dict_pca4_nout, cluster_num=1)

# Cluster Zero

In [95]:
model_dict0 = clusters.get_model_dict(df, cluster_dict=dict_pca4_nout, cluster_num=0)

In [96]:
import cPickle as pickle

In [97]:
pickle.dump(model_dict0, open( "score-predictor/pickles/Cluster0model.p", "wb" ) )

In [98]:
pickle.dump(model_dict1, open( "score-predictor/pickles/Cluster1model.p", "wb" ) )

In [99]:
pickle.dump(model_dict2, open( "score-predictor/pickles/Cluster2model.p", "wb" ) )

In [100]:
pickle.dump(dict_pca4_nout, open( "score-predictor/pickles/cluster_dictionary.p", "wb" ) )

In [ ]:
cluster_dictionary = pickle.load()

In [105]:
filename = 'score-predictor/pickles/cluster_pickles_with_dictionary.r'

In [106]:
with open(filename,'wb') as fp:
    pickle.dump(dict_pca4_nout,fp)
    pickle.dump(model_dict0,fp)
    pickle.dump(model_dict1,fp)
    pickle.dump(model_dict2,fp)

In [107]:
with open(filename,'rb') as fp:
    cl_di=pickle.load(fp)
    model0_d=pickle.load(fp)
    model1_d=pickle.load(fp)
    model2_d=pickle.load(fp)

In [137]:
cl_di

defaultdict(list,
            {0: ['A.J. Price',
              'Aaron Brooks',
              'Aaron McKie',
              'Aaron Miles',
              'Acie Law',
              'Alexey Shved',
              'Allen Iverson',
              'Alvin Williams',
              'Andre Barrett',
              'Andre Iguodala',
              'Andre Miller',
              'Andrei Kirilenko',
              'Anfernee Hardaway',
              'Anthony Carter',
              'Anthony Johnson',
              'Antonio Burks',
              'Antonio Daniels',
              'Armon Johnson',
              'Baron Davis',
              'Ben Hansbrough',
              'Ben Uzoh',
              'Beno Udrih',
              'Bobby Brown',
              'Boris Diaw',
              'Brandon Jennings',
              'Brandon Knight',
              'Brandon Roy',
              'Brevin Knight',
              'Brian Roberts',
              'Briante Weber',
              'C.J. Watson',
              'Cameron Payne',
  

In [133]:
fd = forest_df[forest_df.columns - ['Player Name']]

/Users/ericpei/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  if __name__ == '__main__':


In [146]:
forest_df.columns

Index([u'SP_1dayago', u'SP_2dayago', u'SP_3dayago', u'SP_4dayago',
       u'TP1dayago', u'TP2dayago', u'TP3dayago', u'TP4dayago', u'FG1dayago',
       u'FG2dayago', u'FG3dayago', u'FG4dayago', u'FT1dayago', u'FT2dayago',
       u'FT3dayago', u'FT4dayago', u'TRB1dayago', u'TRB2dayago', u'TRB3dayago',
       u'TRB4dayago', u'AST1dayago', u'AST2dayago', u'AST3dayago',
       u'AST4dayago', u'BLK1dayago', u'BLK2dayago', u'BLK3dayago',
       u'BLK4dayago', u'STL1dayago', u'STL2dayago', u'STL3dayago',
       u'STL4dayago', u'TOV1dayago', u'TOV2dayago', u'TOV3dayago',
       u'TOV4dayago', u'USGP1dayago', u'USGP2dayago', u'USGP3dayago',
       u'USGP4dayago', u'FTr1dayago', u'FTr2dayago', u'FTr3dayago',
       u'FTr4dayago', u'PM1dayago', u'PM2dayago', u'PM3dayago', u'PM4dayago',
       u'TSP1dayago', u'TSP2dayago', u'TSP3dayago', u'TSP4dayago',
       u'PF1dayago', u'PF2dayago', u'PF3dayago', u'PF4dayago', u'ORtg1dayago',
       u'ORtg2dayago', u'ORtg3dayago', u'ORtg4dayago', u'DRtg1dayago'

In [144]:
fd.iloc[50,:]

AST1dayago        1.000
AST2dayago        3.000
AST3dayago        0.000
AST4dayago        1.000
BLK1dayago        0.000
BLK2dayago        0.000
BLK3dayago        0.000
BLK4dayago        0.000
DRtg1dayago     113.000
DRtg2dayago     106.000
DRtg3dayago      97.000
DRtg4dayago     105.000
Elevation       715.000
FG1dayago         4.000
FG2dayago         2.000
FG3dayago         3.000
FG4dayago         4.000
FT1dayago         4.000
FT2dayago         0.000
FT3dayago         0.000
FT4dayago         2.000
FTr1dayago        0.400
FTr2dayago        0.000
FTr3dayago        0.000
FTr4dayago        0.300
ORtg1dayago     106.000
ORtg2dayago     176.000
ORtg3dayago      74.000
ORtg4dayago     108.000
OneisHome         1.000
                 ...   
PM3dayago        -4.000
PM4dayago        -1.000
SP_1dayago        0.000
SP_2dayago     1562.000
SP_3dayago        0.000
SP_4dayago        0.000
STL1dayago        0.000
STL2dayago        0.000
STL3dayago        2.000
STL4dayago        2.000
TOV1dayago      

In [151]:
y_df.loc[50,:]

Player Name    A.J. Price
3P                      5
FG                      6
FT                      0
TRB                     1
AST                     2
BLK                     0
STL                     1
TOV                     2
Name: 50, dtype: object

In [180]:
from scripts import predict_today

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273822 entries, 0 to 273821
Columns: 108 entries, Player Name to Elevation
dtypes: datetime64[ns](5), float64(77), int64(23), object(3)
memory usage: 227.7+ MB


In [177]:
df.loc[50:52,:].iloc[0]['Player Name']

'A.J. Price'

In [190]:
df.loc[50,:].reshape(1, -1)

array([['A.J. Price', 6, 11, 0.54500000000000004, 5, 9,
        0.55600000000000005, 0, 0, 0.0, 0, 1, 1, 2, 1, 0, 2, 1, 17, 12,
        0.77300000000000002, 0.77300000000000002, 0.81799999999999995, 0.0,
        0.0, 4.2000000000000002, 2.5, 11.699999999999999,
        2.1000000000000001, 0.0, 15.4, 24.899999999999999, 130.0, 107.0,
        'HOU', 'IND', 1, Timestamp('2010-04-04 00:00:00'), 1358,
        Timestamp('2010-04-03 00:00:00'), Timestamp('2010-04-02 00:00:00'),
        Timestamp('2010-04-01 00:00:00'), Timestamp('2010-03-31 00:00:00'),
        0, 1562, 0, 0, 1.0, 1.0, 1.0, 2.0, 4.0, 2.0, 3.0, 4.0, 4.0, 0.0,
        0.0, 2.0, 4.0, 1.0, 2.0, 0.0, 1.0, 3.0, 0.0, 1.0, 0.0, 0.0, 0.0,
        0.0, 0.0, 0.0, 2.0, 2.0, 1.0, 0.0, 2.0, 0.0, 24.300000000000001,
        10.800000000000001, 33.5, 30.600000000000001, 0.40000000000000002,
        0.0, 0.0, 0.29999999999999999, 3.0, 0.0, -4.0, -1.0,
        0.55299999999999994, 0.83299999999999996, 0.5, 0.53000000000000003,
        0.0, 0.0,

In [195]:
jj=np.arange(50,270000,2000)

In [194]:
len(df)

273822

In [638]:
df.loc[jj,:]

,Player Name,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,FT%,...,PF4dayago,ORtg1dayago,ORtg2dayago,ORtg3dayago,ORtg4dayago,DRtg1dayago,DRtg2dayago,DRtg3dayago,DRtg4dayago,Elevation
50,A.J. Price,6,11,0.545,5,9,0.556,0,0,0.000,...,2,106,176,74,108,113,106,97,105,715
2050,Adreian Payne,2,2,1.000,0,0,0.000,1,2,0.500,...,2,0,215,104,16,121,125,110,119,830
4050,Al Jefferson,7,19,0.368,0,0,0.000,7,7,1.000,...,4,114,95,139,123,103,94,103,104,830
6050,Alexey Shved,0,1,0.000,0,0,0.000,0,0,0.000,...,0,25,0,99,37,116,124,110,90,830
8050,Amar'e Stoudemire,2,3,0.667,0,0,0.000,0,0,0.000,...,1,125,76,64,133,90,97,112,110,82
10050,Andre Drummond,5,9,0.556,0,0,0.000,3,7,0.429,...,4,110,76,135,118,100,104,95,116,600
12050,Andre Roberson,0,2,0.000,0,2,0.000,0,0,0.000,...,2,124,116,0,121,119,108,109,104,1201
14050,Andrew Bogut,5,9,0.556,0,0,0.000,1,2,0.500,...,3,39,101,104,97,92,104,110,110,33
16050,Antawn Jamison,9,20,0.450,3,6,0.500,3,7,0.429,...,1,130,92,88,123,106,99,87,103,650
18050,Anthony Parker,6,12,0.500,0,4,0.000,0,0,0.000,...,2,142,104,120,107,129,103,119,124,1050


In [196]:
predict_today.predict_now(df.loc[jj,:])

A.J. Price
cluster 0:
[ 3.7] 3P
[ 0.7] FT
[ 2.5] AST
[ 0.7] STL
[ 1.4] TRB
[ 1.2] TOV
[ 0.] BLK
[ 5.2] FG
Adreian Payne
cluster 1:
[ 0.] 3P
[ 0.5] FT
[ 0.] AST
[ 0.1] STL
[ 1.6] TRB
[ 0.2] TOV
[ 0.] BLK
[ 2.] FG
Al Jefferson
cluster 1:
[ 0.] 3P
[ 3.5] FT
[ 2.5] AST
[ 1.] STL
[ 13.] TRB
[ 2.3] TOV
[ 0.4] BLK
[ 10.3] FG
Alexey Shved
cluster 0:
[ 2.8] 3P
[ 4.1] FT
[ 2.2] AST
[ 0.] STL
[ 2.3] TRB
[ 0.3] TOV
[ 0.1] BLK
[ 3.8] FG
Amir Johnson
cluster 1:
[ 1.1] 3P
[ 3.1] FT
[ 1.4] AST
[ 0.] STL
[ 2.2] TRB
[ 1.] TOV
[ 0.1] BLK
[ 4.6] FG
Andre Drummond
cluster 1:
[ 0.] 3P
[ 0.9] FT
[ 0.8] AST
[ 1.2] STL
[ 9.] TRB
[ 1.1] TOV
[ 0.5] BLK
[ 2.6] FG
Andre Roberson
cluster 2:
[ 0.9] 3P
[ 0.] FT
[ 1.] AST
[ 1.] STL
[ 1.2] TRB
[ 1.] TOV
[ 0.] BLK
[ 1.1] FG
Andrew Bogut
cluster 1:
[ 0.] 3P
[ 1.1] FT
[ 1.3] AST
[ 0.1] STL
[ 6.5] TRB
[ 0.4] TOV
[ 1.9] BLK
[ 4.2] FG
Antawn Jamison
cluster 1:
[ 0.2] 3P
[ 0.7] FT
[ 1.4] AST
[ 0.8] STL
[ 1.5] TRB
[ 1.] TOV
[ 0.] BLK
[ 1.6] FG
Anthony Parker
cluster 2:
[ 1.] 3

In [186]:
col_id = []
feature_names = ['SP_1dayago', 'SP_2dayago', 'SP_3dayago', 'SP_4dayago', 'TP1dayago', 'TP2dayago', 'TP3dayago', 'TP4dayago', 
    'FG1dayago', 'FG2dayago', 'FG3dayago', 'FG4dayago', 'FT1dayago', 'FT2dayago', 'FT3dayago', 'FT4dayago', 'TRB1dayago', 'TRB2dayago', 'TRB3dayago', 
    'TRB4dayago', 'AST1dayago', 'AST2dayago', 'AST3dayago', 'AST4dayago', 'BLK1dayago', 'BLK2dayago', 'BLK3dayago', 'BLK4dayago', 'STL1dayago', 
    'STL2dayago', 'STL3dayago', 'STL4dayago', 'TOV1dayago', 'TOV2dayago', 'TOV3dayago', 'TOV4dayago', 'USGP1dayago', 'USGP2dayago', 'USGP3dayago', 
    'USGP4dayago', 'FTr1dayago', 'FTr2dayago', 'FTr3dayago', 'FTr4dayago', 'PM1dayago', 'PM2dayago', 'PM3dayago', 'PM4dayago', 'TSP1dayago', 'TSP2dayago', 
    'TSP3dayago', 'TSP4dayago', 'PF1dayago', 'PF2dayago', 'PF3dayago', 'PF4dayago', 'ORtg1dayago', 'ORtg2dayago', 'ORtg3dayago', 'ORtg4dayago', 'DRtg1dayago', 
    'DRtg2dayago', 'DRtg3dayago', 'DRtg4dayago', 'Elevation', 'OneisHome']
for feat in feature_names:
    col_id.append(np.where(df.loc[50:52,:].columns==feat)[0][0])

print col_id

[43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 36]


In [171]:
a = ['Player Name', 'SP_1dayago', 'SP_2dayago', 'SP_3dayago', 'SP_4dayago', 'TP1dayago', 'TP2dayago', 'TP3dayago', 'TP4dayago', 'FG1dayago', 'FG2dayago', 'FG3dayago', 'FG4dayago', 'FT1dayago', 'FT2dayago', 'FT3dayago', 'FT4dayago', 'TRB1dayago', 'TRB2dayago', 'TRB3dayago', 'TRB4dayago', 'AST1dayago', 'AST2dayago', 'AST3dayago', 'AST4dayago', 'BLK1dayago', 'BLK2dayago', 'BLK3dayago', 'BLK4dayago', 'STL1dayago', 'STL2dayago', 'STL3dayago', 'STL4dayago', 'TOV1dayago', 'TOV2dayago', 'TOV3dayago', 'TOV4dayago', 'USGP1dayago', 'USGP2dayago', 'USGP3dayago', 'USGP4dayago', 'FTr1dayago', 'FTr2dayago', 'FTr3dayago', 'FTr4dayago', 'PM1dayago', 'PM2dayago', 'PM3dayago', 'PM4dayago', 'TSP1dayago', 'TSP2dayago', 'TSP3dayago', 'TSP4dayago', 'PF1dayago', 'PF2dayago', 'PF3dayago', 'PF4dayago', 'ORtg1dayago', 'ORtg2dayago', 'ORtg3dayago', 'ORtg4dayago', 'DRtg1dayago', 'DRtg2dayago', 'DRtg3dayago', 'DRtg4dayago', 'Elevation', 'OneisHome']
a[np.where(a==)]

'Player Name'

In [ ]:
['Player Name', 'SP_1dayago', 'SP_2dayago', 'SP_3dayago', 'SP_4dayago', 'TP1dayago', 'TP2dayago', 'TP3dayago', 'TP4dayago', 'FG1dayago', 'FG2dayago', 'FG3dayago', 'FG4dayago', 'FT1dayago', 'FT2dayago', 'FT3dayago', 'FT4dayago', 'TRB1dayago', 'TRB2dayago', 'TRB3dayago', 'TRB4dayago', 'AST1dayago', 'AST2dayago', 'AST3dayago', 'AST4dayago', 'BLK1dayago', 'BLK2dayago', 'BLK3dayago', 'BLK4dayago', 'STL1dayago', 'STL2dayago', 'STL3dayago', 'STL4dayago', 'TOV1dayago', 'TOV2dayago', 'TOV3dayago', 'TOV4dayago', 'USGP1dayago', 'USGP2dayago', 'USGP3dayago', 'USGP4dayago', 'FTr1dayago', 'FTr2dayago', 'FTr3dayago', 'FTr4dayago', 'PM1dayago', 'PM2dayago', 'PM3dayago', 'PM4dayago', 'TSP1dayago', 'TSP2dayago', 'TSP3dayago', 'TSP4dayago', 'PF1dayago', 'PF2dayago', 'PF3dayago', 'PF4dayago', 'ORtg1dayago', 'ORtg2dayago', 'ORtg3dayago', 'ORtg4dayago', 'DRtg1dayago', 'DRtg2dayago', 'DRtg3dayago', 'DRtg4dayago', 'Elevation', 'OneisHome']

In [150]:
for category, model in model0_d.iteritems():
    print model.predict(fd.loc[50,:][['SP_1dayago', 'SP_2dayago', 'SP_3dayago', 'SP_4dayago', 'TP1dayago', 'TP2dayago', 'TP3dayago', 'TP4dayago', 'FG1dayago', 'FG2dayago', 'FG3dayago', 'FG4dayago', 'FT1dayago', 'FT2dayago', 'FT3dayago', 'FT4dayago', 'TRB1dayago', 'TRB2dayago', 'TRB3dayago', 'TRB4dayago', 'AST1dayago', 'AST2dayago', 'AST3dayago', 'AST4dayago', 'BLK1dayago', 'BLK2dayago', 'BLK3dayago', 'BLK4dayago', 'STL1dayago', 'STL2dayago', 'STL3dayago', 'STL4dayago', 'TOV1dayago', 'TOV2dayago', 'TOV3dayago', 'TOV4dayago', 'USGP1dayago', 'USGP2dayago', 'USGP3dayago', 'USGP4dayago', 'FTr1dayago', 'FTr2dayago', 'FTr3dayago', 'FTr4dayago', 'PM1dayago', 'PM2dayago', 'PM3dayago', 'PM4dayago', 'TSP1dayago', 'TSP2dayago', 'TSP3dayago', 'TSP4dayago', 'PF1dayago', 'PF2dayago', 'PF3dayago', 'PF4dayago', 'ORtg1dayago', 'ORtg2dayago', 'ORtg3dayago', 'ORtg4dayago', 'DRtg1dayago', 'DRtg2dayago', 'DRtg3dayago', 'DRtg4dayago', 'Elevation', 'OneisHome']]), category

[ 3.7] 3P
[ 0.7] FT
[ 2.5] AST
[ 0.7] STL
[ 1.4] TRB
[ 1.2] TOV
[ 0.] BLK
[ 5.2] FG


/Users/ericpei/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/ericpei/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/ericpei/anaconda/lib/python2.7/site-packages/sklearn/utils/validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single s

In [ ]:
model0.predict(forest_df[forest_df].loc[2,:])

In [108]:
cl_di

defaultdict(list,
            {0: ['A.J. Price',
              'Aaron Brooks',
              'Aaron McKie',
              'Aaron Miles',
              'Acie Law',
              'Alexey Shved',
              'Allen Iverson',
              'Alvin Williams',
              'Andre Barrett',
              'Andre Iguodala',
              'Andre Miller',
              'Andrei Kirilenko',
              'Anfernee Hardaway',
              'Anthony Carter',
              'Anthony Johnson',
              'Antonio Burks',
              'Antonio Daniels',
              'Armon Johnson',
              'Baron Davis',
              'Ben Hansbrough',
              'Ben Uzoh',
              'Beno Udrih',
              'Bobby Brown',
              'Boris Diaw',
              'Brandon Jennings',
              'Brandon Knight',
              'Brandon Roy',
              'Brevin Knight',
              'Brian Roberts',
              'Briante Weber',
              'C.J. Watson',
              'Cameron Payne',
  

In [113]:
print len(cl_di[0]), len(cl_di[1]), len(cl_di[2])

254 418 514


# Individual Model

In [197]:
today = datetime.date.today()

In [201]:
today

datetime.date(2016, 4, 20)

In [206]:
yr = today.year
mn = today.month
dy = today.day

In [207]:
print yr, mn, dy

2016 4 20


In [266]:
from scripts import predict_today

In [267]:
str_date

'201604200'

In [848]:
away, home = predict_today.get_url_today_teams()

In [849]:
away

['http://www.basketball-reference.com/teams/BOS/2016.html',
 'http://www.basketball-reference.com/teams/DET/2016.html',
 'http://www.basketball-reference.com/teams/MEM/2016.html',
 'http://www.basketball-reference.com/teams/IND/2016.html']

In [850]:
home

['http://www.basketball-reference.com/teams/ATL/2016.html',
 'http://www.basketball-reference.com/teams/CLE/2016.html',
 'http://www.basketball-reference.com/teams/SAS/2016.html',
 'http://www.basketball-reference.com/teams/TOR/2016.html']

In [851]:
alist, hlist = predict_today.get_list_of_players_today(away, home)

In [630]:
sum(df['Player Name']=='Glenn Robinson III') == 0

True

In [632]:
alist, hlist

([('Dion Waiters', 'DAL'),
  ('Andre Roberson', 'DAL'),
  ('Kevin Durant', 'DAL'),
  ('Russell Westbrook', 'DAL'),
  ('Serge Ibaka', 'DAL'),
  ('Nick Collison', 'DAL'),
  ('Steven Adams', 'DAL'),
  ('Randy Foye', 'DAL'),
  ('Anthony Morrow', 'DAL'),
  ('Mitch McGary', 'DAL'),
  ('Cameron Payne', 'DAL'),
  ('Kyle Singler', 'DAL'),
  ('Enes Kanter', 'DAL'),
  ('Josh Huestis', 'DAL'),
  ('Nazr Mohammed', 'DAL'),
  ('Andrew Bogut', 'HOU'),
  ('Stephen Curry', 'HOU'),
  ('Brandon Rush', 'HOU'),
  ('Harrison Barnes', 'HOU'),
  ('Klay Thompson', 'HOU'),
  ('Festus Ezeli', 'HOU'),
  ('Andre Iguodala', 'HOU'),
  ('Shaun Livingston', 'HOU'),
  ('Marreese Speights', 'HOU'),
  ('James Michael McAdoo', 'HOU'),
  ('Kevon Looney', 'HOU'),
  ('Draymond Green', 'HOU'),
  ('Leandro Barbosa', 'HOU'),
  ('Ian Clark', 'HOU'),
  ('Anderson Varejao', 'HOU'),
  ('Lucas Nogueira', 'IND'),
  ('Patrick Patterson', 'IND'),
  ('DeMar DeRozan', 'IND'),
  ('Kyle Lowry', 'IND'),
  ('Jonas Valanciunas', 'IND'),
  ('Te

In [891]:
played_dict = predict_today.create_line(df,alist,hlist)

In [853]:
played_dict

{'Al Horford': {'AST1dayago': 2,
  'AST2dayago': 2.0,
  'AST3dayago': 2.0,
  'AST4dayago': 2.0,
  'BLK1dayago': 5,
  'BLK2dayago': 2.0,
  'BLK3dayago': 1.0,
  'BLK4dayago': 2.0,
  'DRtg1dayago': 71.0,
  'DRtg2dayago': 103.0,
  'DRtg3dayago': 101.0,
  'DRtg4dayago': 108.0,
  'Elevation': 1050,
  'FG1dayago': 6,
  'FG2dayago': 8.0,
  'FG3dayago': 6.0,
  'FG4dayago': 10.0,
  'FT1dayago': 2,
  'FT2dayago': 7.0,
  'FT3dayago': 5.0,
  'FT4dayago': 0.0,
  'FTr1dayago': 0.182,
  'FTr2dayago': 0.47100000000000003,
  'FTr3dayago': 0.5,
  'FTr4dayago': 0.059000000000000004,
  'ORtg1dayago': 118.0,
  'ORtg2dayago': 121.0,
  'ORtg3dayago': 132.0,
  'ORtg4dayago': 112.0,
  'OneisHome': 1,
  'PF1dayago': 2,
  'PF2dayago': 3.0,
  'PF3dayago': 2.0,
  'PF4dayago': 2.0,
  'PM1dayago': 21,
  'PM2dayago': -4.0,
  'PM3dayago': -10.0,
  'PM4dayago': -8.0,
  'SP_1dayago': 0,
  'SP_2dayago': 0,
  'SP_3dayago': 0,
  'SP_4dayago': 0,
  'STL1dayago': 1,
  'STL2dayago': 0.0,
  'STL3dayago': 1.0,
  'STL4dayago': 1.

In [892]:
played_dict['Isaiah Thomas']

{'AST1dayago': 6,
 'AST2dayago': 1.0,
 'AST3dayago': 2.0,
 'AST4dayago': 8.0,
 'BLK1dayago': 2,
 'BLK2dayago': 0.0,
 'BLK3dayago': 1.0,
 'BLK4dayago': 1.0,
 'DRtg1dayago': 95.0,
 'DRtg2dayago': 108.0,
 'DRtg3dayago': 96.0,
 'DRtg4dayago': 107.0,
 'Elevation': 1050,
 'FG1dayago': 11,
 'FG2dayago': 12.0,
 'FG3dayago': 4.0,
 'FG4dayago': 8.0,
 'FT1dayago': 4,
 'FT2dayago': 13.0,
 'FT3dayago': 7.0,
 'FT4dayago': 7.0,
 'FTr1dayago': 0.217,
 'FTr2dayago': 0.625,
 'FTr3dayago': 0.53299999999999992,
 'FTr4dayago': 0.42899999999999999,
 'ORtg1dayago': 119.0,
 'ORtg2dayago': 135.0,
 'ORtg3dayago': 80.0,
 'ORtg4dayago': 111.0,
 'OneisHome': 0,
 'PF1dayago': 2,
 'PF2dayago': 2.0,
 'PF3dayago': 2.0,
 'PF4dayago': 4.0,
 'PM1dayago': 21,
 'PM2dayago': 3.0,
 'PM3dayago': -16.0,
 'PM4dayago': -5.0,
 'SP_1dayago': 0,
 'SP_2dayago': 2603,
 'SP_3dayago': 0,
 'SP_4dayago': 2199,
 'STL1dayago': 1,
 'STL2dayago': 1.0,
 'STL3dayago': 0.0,
 'STL4dayago': 1.0,
 'TOV1dayago': 1,
 'TOV2dayago': 1.0,
 'TOV3dayago'

In [893]:
predict_today.predict_today(played_dict)

cluster 1:
David West [ 0.3] 3P
David West [ 1.2] FT
David West [ 1.4] AST
David West [ 1.1] STL
David West [ 7.4] TRB
David West [ 1.] TOV
David West [ 0.5] BLK
David West [ 3.2] FG
cluster 0:
Lorenzo Brown [ 0.] 3P
Lorenzo Brown [ 0.5] FT
Lorenzo Brown [ 2.] AST
Lorenzo Brown [ 0.5] STL
Lorenzo Brown [ 2.] TRB
Lorenzo Brown [ 0.5] TOV
Lorenzo Brown [ 0.2] BLK
Lorenzo Brown [ 1.2] FG
cluster 2:
Bruno Caboclo [ 0.2] 3P
Bruno Caboclo [ 0.] FT
Bruno Caboclo [ 0.4] AST
Bruno Caboclo [ 0.2] STL
Bruno Caboclo [ 0.7] TRB
Bruno Caboclo [ 0.2] TOV
Bruno Caboclo [ 0.1] BLK
Bruno Caboclo [ 0.5] FG
cluster 1:
Tim Duncan [ 0.] 3P
Tim Duncan [ 1.3] FT
Tim Duncan [ 2.1] AST
Tim Duncan [ 0.6] STL
Tim Duncan [ 4.5] TRB
Tim Duncan [ 0.8] TOV
Tim Duncan [ 1.6] BLK
Tim Duncan [ 3.2] FG
cluster 1:
Matt Barnes [ 1.5] 3P
Matt Barnes [ 3.2] FT
Matt Barnes [ 3.] AST
Matt Barnes [ 1.2] STL
Matt Barnes [ 10.2] TRB
Matt Barnes [ 1.3] TOV
Matt Barnes [ 0.9] BLK
Matt Barnes [ 5.8] FG
cluster 2:
Jodie Meeks [ 0.9] 

In [480]:
print df.columns[0:100]

Index([u'Player Name', u'FG', u'FGA', u'FG%', u'3P', u'3PA', u'3P%', u'FT',
       u'FTA', u'FT%', u'ORB', u'DRB', u'TRB', u'AST', u'STL', u'BLK', u'TOV',
       u'PF', u'PTS', u'+/-', u'TS%', u'eFG%', u'3PAr', u'FTr', u'ORB%',
       u'DRB%', u'TRB%', u'AST%', u'STL%', u'BLK%', u'TOV%', u'USG%', u'ORtg',
       u'DRtg', u'Away', u'Home', u'OneisHome', u'Date', u'SP', u'DateM1',
       u'DateM2', u'DateM3', u'DateM4', u'SP_1dayago', u'SP_2dayago',
       u'SP_3dayago', u'SP_4dayago', u'TP1dayago', u'TP2dayago', u'TP3dayago',
       u'TP4dayago', u'FG1dayago', u'FG2dayago', u'FG3dayago', u'FG4dayago',
       u'FT1dayago', u'FT2dayago', u'FT3dayago', u'FT4dayago', u'TRB1dayago',
       u'TRB2dayago', u'TRB3dayago', u'TRB4dayago', u'AST1dayago',
       u'AST2dayago', u'AST3dayago', u'AST4dayago', u'BLK1dayago',
       u'BLK2dayago', u'BLK3dayago', u'BLK4dayago', u'STL1dayago',
       u'STL2dayago', u'STL3dayago', u'STL4dayago', u'TOV1dayago',
       u'TOV2dayago', u'TOV3dayago', u'TOV4day

In [835]:
numbers = np.arange(4,40,4)
#numbers = np.arange(40,68,4)

In [824]:
for i in numbers:
    print i, i-4

40 36
44 40
48 44
52 48
56 52
60 56
64 60


In [847]:
sum(model1_d['TOV'].feature_importances_[32:36])

0.052050647833556653

In [834]:
print sum(model1_d['FG'].feature_importances_[:4])
for i in numbers:
    print sum(model1_d['TOV'].feature_importances_[i-4:i])
print model1_d['TOV'].feature_importances_[-2]

0.0567851381599
0.0974185834359
0.0569647501952
0.0802187249324
0.0691809044003
0.0353332906423
0.0725210592149
0.0775698900186
0.0168586042415


In [803]:
model1_d['FG'].feature_importances_

array([ 0.00616044,  0.01540267,  0.01082637,  0.01044037,  0.00170348,
        0.00162148,  0.00185963,  0.00179397,  0.25332904,  0.08735364,
        0.04270027,  0.06269511,  0.00691843,  0.00721573,  0.00626351,
        0.00747747,  0.00961101,  0.00939147,  0.0098464 ,  0.00999197,
        0.00602475,  0.00537894,  0.00577755,  0.00591849,  0.00407443,
        0.00424069,  0.00414922,  0.00396219,  0.00341059,  0.00321175,
        0.00346464,  0.00361672,  0.00519861,  0.00470928,  0.00441136,
        0.00493523,  0.02037146,  0.02454741,  0.02210939,  0.02017782,
        0.00919181,  0.0080498 ,  0.00814928,  0.00840163,  0.01357386,
        0.01379713,  0.01282204,  0.01283252,  0.01379964,  0.01330674,
        0.01289641,  0.01238301,  0.00570525,  0.00545132,  0.00547181,
        0.00576696,  0.01280579,  0.01317282,  0.0121276 ,  0.01240061,
        0.01264445,  0.01365819,  0.013182  ,  0.01279719,  0.01166437,
        0.0016548 ])

In [749]:
sum(model2_d['TOV'].feature_importances_[:4])

0.059220662903760428

In [738]:
model2_d['TOV'].feature_importances_

array([ 0.01110722,  0.01826913,  0.01514336,  0.01470095,  0.00588519,
        0.00588924,  0.00568218,  0.00583141,  0.07781268,  0.02508627,
        0.02488885,  0.01889515,  0.0132283 ,  0.00895587,  0.00937088,
        0.00915504,  0.01197547,  0.01154652,  0.01108109,  0.0119428 ,
        0.01948809,  0.01530341,  0.01501665,  0.01394159,  0.00379617,
        0.00363433,  0.00417409,  0.00389938,  0.00606105,  0.00536797,
        0.00655535,  0.0057238 ,  0.01099509,  0.01291986,  0.01232178,
        0.01226634,  0.02595577,  0.02637118,  0.02523219,  0.02653832,
        0.01302809,  0.01360272,  0.01374352,  0.01360809,  0.02176568,
        0.02118494,  0.02117577,  0.02080874,  0.01951855,  0.01838261,
        0.01817216,  0.01717987,  0.00939642,  0.00827602,  0.00922362,
        0.00907049,  0.01949237,  0.01837125,  0.01840445,  0.01830967,
        0.02162289,  0.02118573,  0.02156113,  0.02082328,  0.01768036,
        0.00240152])

In [661]:
model0_d['FT'].feature_importances_

array([ 0.00860893,  0.01296441,  0.01125693,  0.01062975,  0.00511256,
        0.00488455,  0.00495374,  0.0048584 ,  0.03482939,  0.01727917,
        0.02373179,  0.01241895,  0.07163851,  0.11288545,  0.03809074,
        0.04330674,  0.00798316,  0.00886713,  0.01001031,  0.00741871,
        0.01085539,  0.01071707,  0.01002812,  0.01032378,  0.00290414,
        0.00282398,  0.00238693,  0.00291334,  0.00541366,  0.005721  ,
        0.00520642,  0.00579193,  0.00732756,  0.00730328,  0.00708626,
        0.00698207,  0.03161383,  0.01933949,  0.02394562,  0.02619261,
        0.01463563,  0.01481741,  0.01425922,  0.01291552,  0.01765127,
        0.0177746 ,  0.01632761,  0.0165466 ,  0.01500178,  0.01467449,
        0.01346388,  0.01437335,  0.00607356,  0.00664501,  0.00710599,
        0.00636996,  0.01423591,  0.01408609,  0.01356258,  0.01462053,
        0.01644899,  0.01539638,  0.01614476,  0.0151407 ,  0.01349641,
        0.00162597])

In [662]:
model0_d['3P'].feature_importances_

array([ 0.00933191,  0.01674468,  0.01489273,  0.01336753,  0.09450174,
        0.04353791,  0.02918554,  0.03439092,  0.0090632 ,  0.00917814,
        0.00962883,  0.00934441,  0.00901917,  0.00780072,  0.00861714,
        0.0085089 ,  0.01078038,  0.00901906,  0.01114148,  0.01100129,
        0.01301254,  0.01352919,  0.0121665 ,  0.01264323,  0.00287771,
        0.00266379,  0.00335155,  0.00281273,  0.00653008,  0.0054139 ,
        0.00740642,  0.00594994,  0.00827815,  0.00665885,  0.00689637,
        0.00893606,  0.02127354,  0.02097306,  0.02151239,  0.02047496,
        0.01268751,  0.01386389,  0.01229608,  0.01389587,  0.02087142,
        0.02165752,  0.02200113,  0.02048865,  0.01915087,  0.01992651,
        0.01923389,  0.01947015,  0.00869331,  0.00852503,  0.0083697 ,
        0.00844339,  0.01811303,  0.01844291,  0.01843223,  0.01811742,
        0.02135662,  0.02269056,  0.02018347,  0.02048294,  0.01776951,
        0.00241974])

In [663]:
model0_d['TOV'].feature_importances_

array([ 0.00783704,  0.01552032,  0.01397246,  0.01345357,  0.00537489,
        0.0053896 ,  0.00540327,  0.00536057,  0.10604529,  0.02714553,
        0.01277525,  0.01378171,  0.00799193,  0.00812347,  0.00928365,
        0.00861891,  0.0106184 ,  0.01090341,  0.01225633,  0.01116223,
        0.03514752,  0.03066525,  0.02830545,  0.02462955,  0.00308819,
        0.00294698,  0.00349612,  0.00327124,  0.0051283 ,  0.00608203,
        0.00537923,  0.0061172 ,  0.01063507,  0.01054982,  0.01100934,
        0.00998367,  0.02170776,  0.02395119,  0.02527116,  0.03124183,
        0.01348614,  0.01401197,  0.01381106,  0.01376058,  0.01888279,
        0.01900239,  0.01816227,  0.01884235,  0.01816403,  0.01752931,
        0.01679267,  0.01678913,  0.00841588,  0.00745181,  0.008217  ,
        0.00749353,  0.01693646,  0.01685098,  0.01670577,  0.0170723 ,
        0.01836874,  0.01893021,  0.01906124,  0.01736984,  0.01602057,
        0.00224621])

In [666]:
model2_d['TOV'].feature_importances_

array([ 0.01110722,  0.01826913,  0.01514336,  0.01470095,  0.00588519,
        0.00588924,  0.00568218,  0.00583141,  0.07781268,  0.02508627,
        0.02488885,  0.01889515,  0.0132283 ,  0.00895587,  0.00937088,
        0.00915504,  0.01197547,  0.01154652,  0.01108109,  0.0119428 ,
        0.01948809,  0.01530341,  0.01501665,  0.01394159,  0.00379617,
        0.00363433,  0.00417409,  0.00389938,  0.00606105,  0.00536797,
        0.00655535,  0.0057238 ,  0.01099509,  0.01291986,  0.01232178,
        0.01226634,  0.02595577,  0.02637118,  0.02523219,  0.02653832,
        0.01302809,  0.01360272,  0.01374352,  0.01360809,  0.02176568,
        0.02118494,  0.02117577,  0.02080874,  0.01951855,  0.01838261,
        0.01817216,  0.01717987,  0.00939642,  0.00827602,  0.00922362,
        0.00907049,  0.01949237,  0.01837125,  0.01840445,  0.01830967,
        0.02162289,  0.02118573,  0.02156113,  0.02082328,  0.01768036,
        0.00240152])

In [664]:
model0_d['AST'].feature_importances_

array([ 0.00701914,  0.01549229,  0.01127636,  0.01072447,  0.00433621,
        0.00434618,  0.00467952,  0.00443727,  0.01525283,  0.00949719,
        0.00862595,  0.00857652,  0.00566805,  0.00614999,  0.00569015,
        0.00596357,  0.0074909 ,  0.00757303,  0.00758333,  0.0081637 ,
        0.20085276,  0.08127485,  0.0461634 ,  0.0687886 ,  0.00189164,
        0.00171791,  0.00204222,  0.00186493,  0.00436279,  0.00530751,
        0.00487543,  0.00527593,  0.00693522,  0.00709703,  0.0071226 ,
        0.00729599,  0.01658768,  0.01560807,  0.01678759,  0.01631209,
        0.01051514,  0.01056623,  0.01024356,  0.01098339,  0.01715981,
        0.01467606,  0.01607815,  0.01605683,  0.01459464,  0.01456656,
        0.01464895,  0.01324825,  0.00624544,  0.00682016,  0.00632548,
        0.00618489,  0.01456045,  0.01458326,  0.01357462,  0.01377794,
        0.01489298,  0.0151594 ,  0.01615333,  0.01607373,  0.01287278,
        0.00272709])